# Quantum Eigenvalue Transformation

In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm, sqrtm
import matplotlib.pyplot as plt

In [2]:
X_gate = np.array([[0, 1], [1, 0]], dtype = 'complex_')
Y_gate = np.array([[0, -1j], [1j, 0]], dtype = 'complex_')
Z_gate = np.array([[1, 0], [0, -1]], dtype = 'complex_')

def R_x(theta):
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]], dtype = 'complex_')

def R_y(theta):
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]], dtype = 'complex_')

def R_z(theta):
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]], dtype = 'complex_')

## Hamiltonian

In [3]:
H = X_gate @ X_gate @ Y_gate @ Z_gate + X_gate @ Y_gate @ Z_gate + Y_gate @ Z_gate + Z_gate @ X_gate
print(H)

[[ 0.+1.j  1.+2.j]
 [-1.+2.j  0.+1.j]]


## Block encoding

$$
U =
\begin{bmatrix}
\mathcal{H} & \cdot \\
\cdot & \cdot
\end{bmatrix}
=
\begin{bmatrix}
\mathcal{H} & \sqrt{I - \mathcal{H}^2} \\
\sqrt{I - \mathcal{H}^2} & -\mathcal{H}
\end{bmatrix}
=
Z \otimes \mathcal{H} + \mathcal{H} \otimes \sqrt{\mathcal{I} - \mathcal{H}^2}
$$

In [4]:
def U(H):
    '''
    Returns a block encoding of a Hamiltonian matrix
    '''
    return np.kron(Z_gate, H) + np.kron(X_gate, np.sqrt(np.eye(2) - sqrtm(H)))

In [5]:
print(U(H))

[[ 0.        +1.j          1.        +2.j          0.32831659-0.36996932j
   0.4358115 -0.93136437j]
 [-1.        +2.j          0.        +1.j          0.80632047-0.63813679j
   0.32831659-0.36996932j]
 [ 0.32831659-0.36996932j  0.4358115 -0.93136437j  0.        -1.j
  -1.        -2.j        ]
 [ 0.80632047-0.63813679j  0.32831659-0.36996932j  1.        -2.j
   0.        -1.j        ]]


## Projector

For $\mathcal{H} \in \mathbb{C}^{2 \times 2}$ the projector must be $\Pi =
\begin{bmatrix}
\mathit{I}_2 & \cdot \\
\cdot & \cdot
\end{bmatrix}$

In [6]:
projector = np.block([[np.eye(2), np.zeros((2,2))], [np.zeros((2,2)), np.zeros((2,2))]])

The projector-controlled phase-shift operation is defines ad $\Pi_{\phi} = e^{i\phi (2\Pi-I)}$

In [7]:
def phase_shift(phi):
    '''
    performs the projector-controlled phase shift operation for a given phase phi and a globaly defined projector Pi
    '''
    return expm(1j*phi*(2*projector - np.eye(projector.shape[0])))

## Eigenvalue Transformation

$$
U_{\vec{\phi}} = \prod_{k=1}^{d/2} \Pi_{\phi_{2k-1}} U^{\dagger} \Pi_{\phi_{2k}} U
$$

In [8]:
def eigenval_transform(phi_vec, U):
    '''
    Performs the eigenvalue transformation
    '''
    d = len(phi_vec) - 1
    U_dagger = np.conj(U).T
    if (d % 2 == 0):
        result = 1
        for k in range(1, d//2):
            result = result @ phase_shift(phi_ve[2*k-1]) @ U_dagger @ phase_shift(phi_vec[2*k]) @ U
        return result
    else:
        result = phase_shift(phi_vec[1]) @ U
        for k in range (1, (d-1)//2):
            result = result @ phase_shift(phi_vec[2*k]) @ U_dagger @ phase_shift(phi_vec[2*k+1]) @ U
        return result

## Trivial Example

In [32]:
phi_vec_triv = [0, 0]
transformed = eigenval_transform(phi_vec_triv, U(H))

print(H)
print(projector @ transformed @ projector)

w, v = np.linalg.eig(H)

poly_H = w[0] * v[0].reshape(2, 1) @ v[0].reshape(1, 2).conjugate() + w[1] * v[1].reshape(2, 1) @ v[1].reshape(1, 2).conjugate()
print(poly_H)

[[ 0.+1.j  1.+2.j]
 [-1.+2.j  0.+1.j]]
[[ 0.+1.j  1.+2.j  0.+0.j  0.+0.j]
 [-1.+2.j  0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
[[ 1.66533454e-16+1.j  1.00000000e+00+2.j]
 [-1.00000000e+00+2.j  2.77555756e-17+1.j]]
